# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import math
from collections import Counter
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице представлено 12 столбцов, из них 7 с числовыми значениями, а 5 представляют собой строки. Среди числовых даннных следующие поля: 'children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income'. К строковому типу относятся столбцы: 'education', 'family_status', 'gender','income_type', 'purpose'.Типы данных в столбах в целом соответствуют значениям (кроме столбцов 'days_employed' и 'total_income'), пропущенные значения присутствуют в двух тех же столбцах и их одинаковое количество.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#------------------------------
# Подготока структур для храниения данных
c = df['income_type'].unique()
df['days_employed'] = abs(df['days_employed'])
m_list_for_total_income = []
m_list_for_days_employed = []
#-------------------------------
# Создание словарей из уникальных значений столбцов и соответствующих им медианных значений
for i in c: 
    m_for_total_income = (df[df['income_type']==i]['total_income']).median()
    m_list_for_total_income.append(m_for_total_income)
    m_for_days_employed = (df[df['income_type']==i]['days_employed']).median()
    m_list_for_days_employed.append(m_for_days_employed)
dict_for_total_income = dict(zip(c,m_list_for_total_income))
dict_for_days_employed = dict(zip(c,m_list_for_days_employed))
#-------------------------------
#Функция для замены значений в столбце total_income  
def total_income_without_nan(row):
    total_income = row['total_income']
#    days_employed = row['days_employed']
    income_type = row['income_type']
    if True in list(row.isnull()):
        med_replace_total_income = dict_for_total_income[income_type] # словарь с соответствием медиан 'total_income' и значениями 'income_type' в виде ключей
#        med_replace_days_employed = dict_for_days_employed[income_type] # словарь с соответствием медиан 'days_employed' и значениями 'income_type' в виде ключей
        return med_replace_total_income
#        return med_replace_days_employed
    else:    
        return total_income
#        return days_employed

#------------------------------------
#Функция для замены значений в столбце days_employed (не знаю как вовращать или выводить значения для 2-х столбцов обновременно, поэтому разделил функции)
def days_employed_without_nan(row):
#    total_income = row['total_income']
    days_employed = row['days_employed']
    income_type = row['income_type']
    if True in list(row.isnull()):
#        med_replace_total_income = dict_for_total_income[income_type] # словарь с соответствием медиан 'total_income' и значениями 'income_type' в виде ключей
        med_replace_days_employed = dict_for_days_employed[income_type] # словарь с соответствием медиан 'days_employed' и значениями 'income_type' в виде ключей
#        return med_replace_total_income
        return med_replace_days_employed
    else:    
#        return total_income
        return days_employed

df['total_income'] = df.apply(total_income_without_nan, axis = 1)
df['days_employed'] = df.apply(days_employed_without_nan, axis = 1)  

#-------------------------------        
display(df.head(14))
df.info()



#def without_nan_in_total_income(row):
#    if (row['total_income'].is null()) & (row['income_type'] = 'пенсионер'):
#        df['total_income_nan'] = row['total_income'].fillna(dict_for_income_type['сотрудник'])
#        row['days_employed'] = row['days_employed'].fillna(dict_for_days_employed['income_type'])
#       
#    clients['age_group'] = clients.apply(age_group_unemployed, axis=1) 
#df['total_income'] = df.apply(without_nan, axis=1)
#df['total_income_pens'] = df[df['income_type'] == 'пенсионер']['total_income'].fillna(dict_for_total_income['пенсионер'])
#for i in df['total_income']:
#    if df[df['income_type'] == 'пенсионер'].isnull():
#df['total_income'] = df['total_income'] + df['total_income_pens']
#df['days_employed'] = df[df['income_type'] == 'пенсионер']['days_employed'].fillna(dict_for_days_employed['пенсионер'])
#df['total_income'] = df[df['income_type'] == 'сотрудник']['total_income'].fillna(dict_for_total_income['сотрудник'])
#df['days_employed'] = df[df['income_type'] == 'сотрудник']['days_employed'].fillna(dict_for_days_employed['сотрудник'])
#for i in range(df.shape[0]):
#    if (df.loc[i,'total_income'] == & (df.loc[i,'income_type'] == c): 
#        df.loc[i,'income_type'] = df.loc[i,'income_type'].replace('0',str(dict_for_income_type[df.loc[i,'income_type']]))
#        df.loc[i,'days_employed'] = df.loc[i,'days_employed'].replace('0',str(dict_for_days_employed[df.loc[i,'days_employed']]))
# Замена пропущенных значений на медианные
#for i in range(df.shape[0]):
#    if (df.loc[i,'total_income'].isnull()) & (df.loc[i,'income_type'] in c): 
#        df.loc[i,'income_type'] = df.loc[i,'income_type'].replace('0',str(dict_for_income_type[df.loc[i,'income_type']]))
#        df.loc[i,'total_income'] = dict_for_income_type[df.loc[i,'income_type']]
#    if (df.loc[i,'days_employed'] == 0) & (df.loc[i,'dob_years'] in d): 
#        df.loc[i,'days_employed'] = dict_for_days_employed[df.loc[i,'dob_years']]
#---------------------------


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Для избавления от пропущенных значений пришлось удалить 2173 строки, что может повлиять на результат исследования (~10% от всего объема данных). Медианные значения распределенные по возрастам заемщиков дали бы более достоверный результат. Тем не менее основная задача выполнена. Предположительно данные пропуски появились из-за отсутствия необходимости заполненения полей  'days_employed' и 'total_income' для определенных кредитов. При этом данные ропуки изначально не обрабатывались.

P.S. Не обрабатывал артефакты (выбросы) в столбце 'days_employed', как лучше с ними поступить: раделить на продолжительность рабочего дня (8 часов) или на продолжительность суток - 24 часа?

### Замена типа данных

In [11]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

### Вывод

Произведена замена типа 'foat' на  тип 'int' в столбце со значениями трудового стажа в днях, поскольку оно является целочисленным значением. Применен метод astype() т.к. в нем можно явно указать тип, в который преобразуются значения. Еще на предыдущем шаге убраны отрицательные значения в столбце с трудовым стажем.

### Обработка дубликатов

In [12]:
#print(df['education'].value_counts())
#print(df['income_type'].value_counts())
#print(df['family_status'].value_counts())
#education +
#income_type -
#family_status +
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
#display(df.groupby(['days_employed', 'total_income'])['purpose'].count())
df = df.drop_duplicates().reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21120 entries, 0 to 21119
Data columns (total 13 columns):
children            21120 non-null int64
days_employed       21120 non-null int64
dob_years           21120 non-null int64
education           21120 non-null object
education_id        21120 non-null int64
family_status       21120 non-null object
family_status_id    21120 non-null int64
gender              21120 non-null object
income_type         21120 non-null object
debt                21120 non-null int64
total_income        21120 non-null int64
purpose             21120 non-null object
income_level        21120 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

Проведя анализ столбцов, определил, что повторящиеся данные могут находиться в столбцах 'education', 'family_status', 'income_type'. В итоге произвел замену данных в столбце 'education'. Нахождение дубликатов производил методом value_counts(). В столбце 'family_status' методом str.lower() привел все значения к одному формату. Предположительно повторы появились при заполнении формы на кредит либо сотрудникамии банка, либо клиентами, при этом изначально не был определен формат заполнения данных полей. Дубликаты обнаружены после замены пропущенных значений (а не их удаления). В результате из изначального массива данных удалено 405 записей.

### Лемматизация

In [13]:
list_for_purpose_lemms = []
for i in range(df.shape[0]):
    lemmas = m.lemmatize(df.loc[i, 'purpose'])
    list_for_purpose_lemms += lemmas
counter = Counter(list_for_purpose_lemms)    
print(counter.most_common(len(counter)))

[('\n', 21120), ('недвижимость', 10578), ('автомобиль', 4272), ('образование', 3964), ('свадьба', 2306)]


### Вывод

Лемматизация данных производилась по столбцу 'purpuse' путем нахождения лемм в каждой записи, затем данные записывались в общий список, производился подсчет количества уникальных значений в этом списке и сортировка их количества по убыванию в целях более наглядного представления и удобного анализа. Наиболее часто в целях кредита встречаются леммы, характеризующие действия с жильем/недвижимостью. 

### Категоризация данных

In [6]:
#--------------------------------------
#Категоризация по наиболее популярным леммам
def purpose_group(purpose_string):
    lemmas = m.lemmatize(purpose_string)
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'иная цель'
df['purpose'] = df['purpose'].apply(purpose_group)
income_median = df['total_income'].median()
def income_level(income):
    if 0 < income <= 0.7*income_median:
        return 'низкий'
    elif 0.7*income_median <= income <= 1.15*income_median:
        return 'ниже среднего'
    elif 1.15*income_median < income <= 1.55*income_median:
        return 'выше среднего'
    elif 1.55*income_median < income:
        return 'высокий'
df['income_level'] = df['total_income'].apply(income_level)
display(df.head(15))
#------------------------------------
# Выделение словарей
c = df[['education_id', 'education']]
d = df[['family_status_id', 'family_status']]
for i in c:
    dict_education = dict(zip(c.loc[:,'education_id'], c.loc[:,'education']))
    dict_family_status = dict(zip(d.loc[:,'family_status_id'], d.loc[:,'family_status']))
print(dict_education)    
print(dict_family_status)           

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,ниже среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,ниже среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,ниже среднего


{0: 'высшее', 1: 'среднее', 2: 'неоконченное высшее', 3: 'начальное', 4: 'ученая степень'}
{0: 'женат / замужем', 1: 'гражданский брак', 2: 'вдовец / вдова', 3: 'в разводе', 4: 'не женат / не замужем'}


### Вывод

В данном задании я ввел 2 словаря. Первый из них классифицирует клиентов по цели кредита и заменяет значения столбца 'purpose' более общими, деля их на 4 категории: 'недвижимость', 'автомобиль', 'образование', ' свадьба'. Нераспределенных целей кредитов не обнаружено. Второй словарь введен для разделения кредитуемых лиц по уровню дохода относительно медианного значения столбца 'total_income': 'низкий', 'ниже среднего', 'выше среднего', 'высокий'. Оба словаря введены с целью ответа на вопросы шага 3. 

P.S. В комментариях в сообществе slackа приводились данные исследования ВШЭ о распределении людей на категории по уровню дохода. Правда там приводились 3 группы, я немного разнес границы средней части и разделил ее на 2 ('ниже среднего' и ' выше среднего').

P.S.S Не думал, что здесь имеются ввиду 'словари' как структура данных языка python))

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data_pivot = df.pivot_table(index = ['children'], columns = 'debt',  values = 'purpose', aggfunc = 'count')
data_pivot = data_pivot.fillna(0)
data_pivot['Возвращаемость'] = data_pivot[0]/(data_pivot[0] + data_pivot[1])
stat_with_children = df[df['children'] != 0].groupby('debt')['purpose'].count()
with_children = stat_with_children[0]/(stat_with_children[0] + stat_with_children[1])
print('Возвращаемость кредитов клиентами с детьми составляет', with_children, '\n')
print(data_pivot)

Возвращаемость кредитов клиентами с детьми составляет 0.9079179682194758 

debt            0       1  Возвращаемость
children                                 
0         13028.0  1063.0        0.924562
1          4410.0   445.0        0.908342
2          1926.0   202.0        0.905075
3           303.0    27.0        0.918182
4            37.0     4.0        0.902439
5             9.0     0.0        1.000000


### Вывод

 Оценка влияния количества детей в семье на возращаемость кредитов позволяет сделать вывод о том, что бездетные люди вероятнее осуществят выплату по займу в срок (92,5%), чем это сделают люди с детьми (90.7%). Предположительно данная тенденция связана с частыми непредвиденными тратами на детей и дополнительной статьей расхода. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_pivot_1 = df.pivot_table(index = ['family_status'], columns = 'debt',  values = 'purpose', aggfunc = 'count')
#data_pivot_1 = data_pivot_1.fillna(0)
data_pivot_1['Возвращаемость'] = data_pivot_1[0]/(data_pivot_1[0] + data_pivot_1[1])
print(data_pivot_1.sort_values(by = 'Возвращаемость', ascending = False))

debt                       0    1  Возвращаемость
family_status                                    
вдовец / вдова           896   63        0.934307
в разводе               1110   85        0.928870
женат / замужем        11408  931        0.924548
гражданский брак        3763  388        0.906529
не женат / не замужем   2536  274        0.902491


### Вывод

 По результатам анализа удалось установить, что наибольшее влияние на возвращаемость кредита в срок оказывает семейное положение и цель кредита. Таким образом вдовцы\вдовы и люди, находящиеся в разводе, по статистике возвращают кредиты более ответственно, чем остальные группы клиентов банка (93,4% и 92,9%). Однако в сумме 2 эти группы в сумме составляют всего 10% от рассматриваемого количества людей. В тоже время клиенты, находящиеся замужем / женатые, сотавляют более 50% всего массива данных и возвращаемость кредитов в срок данной категорией лиц не на много ниже, чем у двух предыдущих (92,5%). Вероятно, это связано с принятием решения о необходимости кредита не одним человеком, а целой семьей, что позволяет более разносотронне определять финансовые риски. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_pivot_2 = df.pivot_table(index = ['income_level'], columns = 'debt',  values = 'purpose', aggfunc = 'count')
#data_pivot_2 = data_pivot_1.fillna(0)
data_pivot_2['Возвращаемость'] = data_pivot_2[0]/(data_pivot_2[0] + data_pivot_2[1])
print(data_pivot_2.sort_values(by = 'Возвращаемость', ascending = False))

debt              0    1  Возвращаемость
income_level                            
высокий        3681  280        0.929311
низкий         4086  353        0.920478
выше среднего  3981  359        0.917281
ниже среднего  7965  749        0.914046


### Вывод

Возвращаемость кредитов клиентами c высоким уровнем дохода незначительно выше, чем данный показатель у остальных категорий кредитуемых лиц. Данная закономерность связана с их более высокой платежной способностью. 

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
data_pivot_3 = df.pivot_table(index = ['purpose'], columns = 'debt',  values = 'income_level', aggfunc = 'count')
data_pivot_3['Возвращаемость'] = data_pivot_3[0]/(data_pivot_3[0] + data_pivot_3[1])
print(data_pivot_3.sort_values(by = 'Возвращаемость', ascending = False))

debt              0    1  Возвращаемость
purpose                                 
недвижимость  10029  782        0.927666
свадьба        2138  186        0.919966
образование    3643  370        0.907800
автомобиль     3903  403        0.906410


### Вывод

Возвращаемость кредитов на недвижимость и свадьбы выше, чем на образование и автомобили. Данная ситуация может быть связана с тем, что к покупке недвижимости люди подходят более ответственно из-за её высокой стоимости, а свадьбы зачастую окупаются за счет подарков.    

### Шаг 4. Общий вывод

   В ходе анализа предоставленных данных проведена оценка зависимости между различными факторами (семейное положение, уровень дохода, наличие и количество детей, цель кредита) и возвращаемостью кредита в срок. Осуществлена предобработка данных, а также классификация кредетуемых лиц по группам для более качественного определения влияния рассматриваемых показателей.  
    По результатам анализа удалось установить,что клиенты, находящиеся замужем / женатые, сотавляют более 50% всего массива данных и возвращаемость кредитов в срок данной категорией лиц (92,5%) не на много ниже, чем у вдовцы\вдовы и люди, находящиеся в разводе, вдовцы\вдовы и люди, находящиеся в разводе (93,4% и 92,9%). Вероятно, это связано с принятием решения о необходимости кредита не одним человеком, а целой семьей, что позволяет более разносторонне определять финансовые риски.      
    Возвращаемость кредитов на недвижимость (92,8%) и свадьбы (92,0%) выше, чем на образование (90,8%) и автомобили (90,6%). Данная ситуация может быть связана с тем, что к покупке недвижимости люди подходят более ответственно из-за её высокой стоимости, а свадьбы зачастую окупаются за счет подарков.
    Оценка влияния количества детей в семье на возращаемость кредитов позволяет сделать вывод о том, что бездетные люди вероятнее осуществят выплату по займу в срок (92,5%), чем это сделают люди с детьми (90.7%). Предположительно данная тенденция связана с частыми непредвиденными тратами на детей и дополнительной статьей расхода.
    Возвращаемость кредитов клиентами c высоким уровнем дохода незначительно выше (92,9%), чем данный показатель у остальных категорий кредитуемых лиц (91,6%). Данная закономерность связана с их более высокой платежной способностью. 
    

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [X]  открыт файл;
- [X]  файл изучен;
- [X]  определены пропущенные значения;
- [X]  заполнены пропущенные значения;
- [X]  есть пояснение, какие пропущенные значения обнаружены;
- [X]  описаны возможные причины появления пропусков в данных;
- [X]  объяснено, по какому принципу заполнены пропуски;
- [X]  заменен вещественный тип данных на целочисленный;
- [X]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [X]  удалены дубликаты;
- [X]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [X]  описаны возможные причины появления дубликатов в данных;
- [X]  выделены леммы в значениях столбца с целями получения кредита;
- [X]  описан процесс лемматизации;
- [X]  данные категоризированы;
- [X]  есть объяснение принципа категоризации данных;
- [X]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [X]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [X]  в каждом этапе есть выводы;
- [X]  есть общий вывод.